In [7]:
import asyncio
from playwright.async_api import async_playwright
import imapclient
import pyzmail

In [8]:
async def fetch_otp_from_email():
    with imapclient.IMAPClient('imap.gmail.com') as client:
        client.login('your_email@example.com', 'your_password')
        client.select_folder('INBOX')
        messages = client.search(['UNSEEN'])
        for msgid, data in client.fetch(messages, 'RFC822').items():
            email_message = pyzmail.PyzMessage.factory(data[b'RFC822'])
            if 'Your OTP' in email_message.get_subject():
                otp_code = extract_otp_from_email(email_message.text_part.get_payload().decode('utf-8'))
                return otp_code

In [9]:
def extract_otp_from_email(email_content):
    otp_code = email_content.strip()  # Simplistic, refine as needed
    return otp_code

In [10]:
async def run(playwright):
    browser = await playwright.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto('https://prenotami.esteri.it/UserArea')

    # Login process
    await page.fill('#email', 'your_email@example.com')
    await page.click('#request_otp')

    # Fetch OTP from email
    otp_code = await fetch_otp_from_email()
    await page.fill('#otp', otp_code)
    await page.click('#submit')

    # Navigate to reservation section
    await page.click('a[href="/ReservationLink"]')
    await page.click('a:has-text("Third Option")')

    # Check for availability
    if await page.is_visible('.no-availability-popup'):
        print("No availability found.")
    else:
        print("Availability found, selecting date.")
        available_date = await page.query_selector('.calendar-green')
        await available_date.click()

    await browser.close()

In [11]:
async def main():
    async with async_playwright() as playwright:
        await run(playwright)

In [12]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop